## Single Model Approach
This approach uses a single Consumption model to compute the prediction.
Taking as input Latitude, Longitude, Month and Lane Count

In [5]:
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load model
consumption_model_path = "../../Model/Exports/Consumption Model/random_forest_consumption_model_location_lane_month.pkl"
consumption_model = joblib.load(consumption_model_path)

def predict_consumption(lat, lon, lane_count, month):
    query = pd.DataFrame([{
        "Latitude": lat,
        "Longitude": lon,
        "Lane Count": lane_count,
        "Month": month,
    }])

    return float(consumption_model.predict(query)[0])

def predict_consumption_year(lat, lon, lane_count):
    monthly_consumption = []
    total = 0
    for i in range(1, 13):
        consumption = predict_consumption(lat, lon, lane_count, i)
        total += consumption
        monthly_consumption.append(consumption)
    return {"Total": total, "Monthly": monthly_consumption}

# Performs a test from an input dataset following the clean format
def test_model(data):
    X = data[["Latitude", "Longitude", "Lane Count", "Month"]]
    y_true = data["Estimated Monthly Consumption (kWh)"]
    
    # Make predictions row by row
    y_pred = X.apply(lambda row: predict_consumption(
        row["Latitude"], row["Longitude"], row["Lane Count"], row["Month"]
    ), axis=1)

    # Compute evaluation metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return {
        "X": X,
        "Y": y_true,
        "Prediction": y_pred,
        "MAE": mae,
        "MSE": mse,
        "R2": r2
    }


## Run

In [6]:
# Example run
month = 4
consumption = predict_consumption(125.07623, 121.419053, 3, month=month)

print(consumption)

1178.4033333333334


## Test

In [8]:
path = "../../Clean Data/Clean_Electricity_Data_With_Cost.csv"
df = pd.read_csv(path)

print(test_model(df)["R2"])

0.9244225551958796


# Estimate Cost

In [2]:
import sys
sys.path.append('Cost_Estimation.py')
import Const_Estimation as cost

import importlib 
importlib.reload(cost)


<module 'Const_Estimation' from 'd:\\Power_Consumption_Machine_Learning\\PowerConsumtionPrediction\\Experimentation & Visualization\\Pipelines\\Const_Estimation.py'>

In [8]:
estimated_cost = cost.calculate_electricity_bill(month=month, consumption_kwh=consumption)

print("Estimated Cost: ", estimated_cost)

Estimated Cost:  3479.89
